Facciamo delle prove a convertire file e preprocessare!

In [ ]:
import os
from pdf2image import convert_from_path
from tqdm import trange

input_file ="../data/raw/full/BerolPhill1516.pdf"
output_path="../data/raw/pages/BerolPhill1516"
os.makedirs(output_path, exist_ok=True)

dpi=350
first_page=13
last_page=23
#last_page=396

for i in trange(first_page, last_page + 1, desc="Converting pages"):
    pages = convert_from_path(
        input_file,
        dpi=dpi,
        first_page=i,
        last_page=i,
        fmt="png"
    )
    page = pages[0]
    out_path = os.path.join(output_path, f"page_{i:03d}.png")
    page.save(out_path, "PNG")

#USA POPPLER BRUTTO DISGRAZIATO

Converting pages:  36%|███▋      | 4/11 [02:27<04:18, 36.93s/it]


KeyboardInterrupt: 

In [14]:
import cv2
import os


input_dir = "../data/raw/pages/BerolPhill1516"
gsc_dir = "../data/processed/gsc"
bzd_dir = "../data/processed/bzd"
ctr_dir = "../data/processed/ctr"

os.makedirs(gsc_dir,exist_ok=True)
os.makedirs(bzd_dir,exist_ok=True)
os.makedirs(ctr_dir,exist_ok=True)

images = sorted([f for f in os.listdir(input_dir) if f.endswith(".png")])

print(f"We will convert {len(images)} pages!!")

for name in tqdm(images, desc="STO LAVORANDO"):
    path = os.path.join(input_dir, name)

    gsc_path = os.path.join(gsc_dir, name)
    bzd_path = os.path.join(bzd_dir, name)
    ctr_path = os.path.join(ctr_dir, name)

    gsc_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    #save gsc
    cv2.imwrite(gsc_path, gsc_img)

    _, bzd_img = cv2.threshold(
        gsc_img, 
        0,
        255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    #save bzd
    cv2.imwrite(bzd_path, bzd_img)

    blurred = cv2.GaussianBlur(gsc_img, (5,5), 0)
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)
    edges_inv = cv2.bitwise_not(edges)

    #sace ctr
    cv2.imwrite(ctr_path, edges_inv)

We will convert 14 pages!!


STO LAVORANDO:  64%|██████▍   | 9/14 [00:15<00:08,  1.76s/it]


error: OpenCV(4.12.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:1146: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
